# 5. Preset Pass Manager
References:
- https://docs.quantum.ibm.com/transpile/transpiler-stages


### Generate different circuit types

In [ ]:
from qiskit.circuit.random import random_circuit
from qiskit.circuit.library import TwoLocal
import random
import numpy as np

num_qubits_list = [5, 10, 20]
depth_list = [5, 10, 20]

# use this for now
num_qubits = num_qubits_list[-1]
depth = depth_list[-1]

# Random circuits
random_circuit_list  = [random_circuit(num_qubits=num_qubits, depth=depth).decompose().decompose()]

# Dense layered circuits
parametrized_dense_layered_circuit_list = [TwoLocal(num_qubits=num_qubits, rotation_blocks='rx', entanglement_blocks='cx',  reps=depth, entanglement='pairwise').decompose().decompose()]

# Staircase circuits
parametrized_staircase_circuit_list = [TwoLocal(num_qubits=num_qubits, rotation_blocks='rx', entanglement_blocks='cx',  reps=depth, entanglement='linear').decompose().decompose()]

#Assign parameters and measurements
dense_layered_circuit_list, staircase_circuit_list = [], []
for dense_circuit, staircase_circuit in zip(parametrized_dense_layered_circuit_list,parametrized_staircase_circuit_list):
    num_params_dense = dense_circuit.num_parameters; num_params_stair = staircase_circuit.num_parameters
    values_dense = [random.uniform(0, 2*np.pi) for _ in range(num_params_dense)]; values_stair = [random.uniform(0, 2*np.pi) for _ in range(num_params_stair)]
    dense_circuit.assign_parameters(values_dense, inplace=True);  staircase_circuit.assign_parameters(values_stair, inplace=True)
    dense_circuit.measure_all(); staircase_circuit.measure_all()
    dense_layered_circuit_list.append(dense_circuit.decompose().decompose()); staircase_circuit_list.append(staircase_circuit.decompose().decompose())

for circuit in random_circuit_list:
    circuit.measure_all()




In [ ]:
print('# CX:', random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth:',random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2) )
random_circuit_list[0].draw(fold=-1)

In [ ]:
print('# CX:', dense_layered_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth:',dense_layered_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2) )
dense_layered_circuit_list[0].draw(fold=-1)

In [ ]:
print('# CX:', staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth:',staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2) )
staircase_circuit_list[0].draw(fold=-1)

## Optimize circuit for hardware topology (routing+optimizations)

### Define hardware topology

In [ ]:
from qiskit.providers.fake_provider import Fake20QV1
from qiskit.transpiler import CouplingMap

backend = Fake20QV1()

# Get a coupling map
coupling_map_list = backend.configuration().coupling_map
coupling_map_obj = CouplingMap(coupling_map_list)


# Get supported basis gates
basis_gates = backend.configuration().basis_gates

#Backend properties
backend_properties = backend.properties()

print('basis:', basis_gates)
coupling_map_obj.draw()

### Simple mapping
Let's not dwell on mapping for now. This is an important topic that we'll tackle later

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import TrivialLayout, FullAncillaAllocation, ApplyLayout

# Enlarge circuit to include physical qubits
set_layout = TrivialLayout(coupling_map_obj)
allocate_ancillas = FullAncillaAllocation(coupling_map_obj)
layout = ApplyLayout()

pm = PassManager([set_layout, allocate_ancillas, layout])

lt_random_circuit_list = pm.run(random_circuit_list)
lt_dense_circuit_list = pm.run(dense_layered_circuit_list)
lt_staircase_circuit_list = pm.run(staircase_circuit_list)


In [ ]:
print('Random circuits')
print('# CX before:', random_circuit_list[0].decompose().count_ops()['cx'], ', after:', lt_random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  lt_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))
# lt_random_circuit_list[0].draw(fold=-1)

print('')
print('Dense circuits')
print('# CX before:', dense_layered_circuit_list[0].decompose().count_ops()['cx'], ', after:', lt_dense_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', dense_layered_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  lt_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', lt_staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  lt_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))


### Routing
This is a necessary step to map virtual circuit to hardware topology, most likely the number of 2Q gates will increase

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import SabreSwap

trials = 100
sabre_routing = SabreSwap(coupling_map=coupling_map_obj, trials=trials, heuristic='lookahead')

pm = PassManager([sabre_routing])

rt_random_circuit_list = pm.run(lt_random_circuit_list)
rt_random_circuit_list = [circuit.decompose() for circuit in rt_random_circuit_list]

rt_dense_circuit_list = pm.run(lt_dense_circuit_list)
rt_dense_circuit_list = [circuit.decompose() for circuit in rt_dense_circuit_list]

rt_staircase_circuit_list = pm.run(lt_staircase_circuit_list)
rt_staircase_circuit_list = [circuit.decompose() for circuit in rt_staircase_circuit_list]



In [ ]:
print('Random circuits')
print('# CX before:', lt_random_circuit_list[0].decompose().count_ops()['cx'], ', after:', rt_random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', lt_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  rt_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))
# lt_random_circuit_list[0].draw(fold=-1)

print('')
print('Dense circuits')
print('# CX before:', lt_dense_circuit_list[0].decompose().count_ops()['cx'], ', after:', rt_dense_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', lt_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  rt_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', lt_staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', rt_staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', lt_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  rt_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))


### Collect

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Collect2qBlocks, ConsolidateBlocks, Collect1qRuns

approximation_degree = 0.99
collect_2q = Collect2qBlocks()
collect_1q = Collect1qRuns()
consolidate = ConsolidateBlocks(basis_gates=basis_gates, approximation_degree=approximation_degree)

pm = PassManager([collect_2q, collect_1q, consolidate])

cl_random_circuit_list = pm.run(rt_random_circuit_list)

cl_dense_circuit_list = pm.run(rt_dense_circuit_list)

cl_staircase_circuit_list = pm.run(rt_staircase_circuit_list)

In [ ]:
print('Random circuits')
print('# CX before:', rt_random_circuit_list[0].decompose().count_ops()['cx'], ', after:', cl_random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', rt_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  cl_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))
# lt_random_circuit_list[0].draw(fold=-1)

print('')
print('Dense circuits')
print('# CX before:', rt_dense_circuit_list[0].decompose().count_ops()['cx'], ', after:', cl_dense_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', rt_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  cl_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', rt_staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', cl_staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', rt_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  cl_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))


### Synthesis
This step synthesize gates according to some basis

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import UnitarySynthesis, HighLevelSynthesis, BasisTranslator
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel

approximation_degree = 0.99
natural_direction = False
un_synthesis = UnitarySynthesis(basis_gates = basis_gates, 
                             approximation_degree=approximation_degree,
                             coupling_map=coupling_map_obj,
                             backend_props = backend_properties,
                             natural_direction=natural_direction
                             )

hl_synthesis = HighLevelSynthesis(coupling_map=coupling_map_obj,
                                    basis_gates=basis_gates,
                                    use_qubit_indices=True,
                                    equivalence_library=sel,
                                    min_qubits=3)

basis_trans = BasisTranslator(sel, basis_gates, min_qubits=3)

pm = PassManager([un_synthesis, hl_synthesis, basis_trans])

synth_random_circuit_list = pm.run(cl_random_circuit_list)
synth_dense_circuit_list = pm.run(cl_dense_circuit_list)
synth_staircase_circuit_list = pm.run(cl_staircase_circuit_list)

In [ ]:
print('Random circuits')
print('# CX before:', cl_random_circuit_list[0].decompose().count_ops()['cx'], ', after:', synth_random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', cl_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  synth_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))
# lt_random_circuit_list[0].draw(fold=-1)

print('')
print('Dense circuits')
print('# CX before:', cl_dense_circuit_list[0].decompose().count_ops()['cx'], ', after:', synth_dense_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', cl_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  synth_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', cl_staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', synth_staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', cl_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  synth_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))


### Optimization

#### Hoare optimization

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import HoareOptimizer

hoare_opt = HoareOptimizer()

pm = PassManager([hoare_opt])

ho_random_circuit_list = pm.run(synth_random_circuit_list)
ho_dense_circuit_list = pm.run(synth_dense_circuit_list)
ho_staircase_circuit_list = pm.run(synth_staircase_circuit_list)

In [ ]:
# ho_random_circuit_list = synth_random_circuit_list
# ho_dense_circuit_list = synth_dense_circuit_list
# ho_staircase_circuit_list = synth_staircase_circuit_list

In [ ]:
print('Random circuits')
print('# CX before:', synth_random_circuit_list[0].decompose().count_ops()['cx'], ', after:', ho_random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', synth_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  ho_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))
# lt_random_circuit_list[0].draw(fold=-1)

print('')
print('Dense circuits')
print('# CX before:', synth_dense_circuit_list[0].decompose().count_ops()['cx'], ', after:', ho_dense_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', synth_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  ho_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', synth_staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', ho_staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', synth_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  ho_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))


#### Simplify through commutations

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import CommutationAnalysis, CommutativeCancellation, CommutativeInverseCancellation, Optimize1qGatesSimpleCommutation

comm_analysis = CommutationAnalysis()
comm_1q = Optimize1qGatesSimpleCommutation()
comm_canc = CommutativeCancellation(basis_gates=basis_gates)
comm_inv_canc = CommutativeInverseCancellation(matrix_based = True, max_qubits = 4)

pm = PassManager([comm_analysis,comm_1q, comm_canc, comm_inv_canc])

comm_random_circuit_list = pm.run(ho_random_circuit_list)
comm_dense_circuit_list = pm.run(ho_dense_circuit_list)
comm_staircase_circuit_list = pm.run(ho_staircase_circuit_list)

In [ ]:
print('Random circuits')
print('# CX before:', ho_random_circuit_list[0].decompose().count_ops()['cx'], ', after:', comm_random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', ho_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  comm_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))
# lt_random_circuit_list[0].draw(fold=-1)

print('')
print('Dense circuits')
print('# CX before:', ho_dense_circuit_list[0].decompose().count_ops()['cx'], ', after:', comm_dense_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', ho_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after', comm_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', ho_staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', comm_staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', ho_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  comm_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))


#### Optimize consecutive gates

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Optimize1qGates, Optimize1qGatesDecomposition, CXCancellation, InverseCancellation
from qiskit.circuit.library.standard_gates import (
    CXGate,
    ECRGate,
    CZGate,
    XGate,
    YGate,
    ZGate,
    TGate,
    TdgGate,
    SwapGate,
    SGate,
    SdgGate,
    HGate,
    CYGate,
    SXGate,
    SXdgGate,
)


gates_to_cancel = [
                        CXGate(),
                        ECRGate(),
                        CZGate(),
                        CYGate(),
                        XGate(),
                        YGate(),
                        ZGate(),
                        HGate(),
                        SwapGate(),
                        (TGate(), TdgGate()),
                        (SGate(), SdgGate()),
                        (SXGate(), SXdgGate()),
                    ]

opt_1q = Optimize1qGates()
opt_1q_dec = Optimize1qGatesDecomposition()
cx_canc = CXCancellation()
inv_canc = InverseCancellation(gates_to_cancel=gates_to_cancel)

pm = PassManager([opt_1q, opt_1q_dec, cx_canc, inv_canc])

opt_random_circuit_list = pm.run(comm_random_circuit_list)
opt_dense_circuit_list = pm.run(comm_dense_circuit_list)
opt_staircase_circuit_list = pm.run(comm_staircase_circuit_list)

In [ ]:
print('Random circuits')
print('# CX before:', comm_random_circuit_list[0].decompose().count_ops()['cx'], ', after:', opt_random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', comm_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  opt_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))
# lt_random_circuit_list[0].draw(fold=-1)

print('')
print('Dense circuits')
print('# CX before:', comm_dense_circuit_list[0].decompose().count_ops()['cx'], ', after:', opt_dense_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', comm_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after', opt_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', comm_staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', opt_staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', comm_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  opt_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))


### Optimize gates preceding a measurement

In [ ]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import OptimizeSwapBeforeMeasure, RemoveDiagonalGatesBeforeMeasure

opt_swap_meas = OptimizeSwapBeforeMeasure()
opt_diag_meas = RemoveDiagonalGatesBeforeMeasure()

pm = PassManager([opt_swap_meas, opt_diag_meas])

opt_meas_random_circuit_list = pm.run(opt_random_circuit_list)
opt_meas_dense_circuit_list = pm.run(opt_dense_circuit_list)
opt_meas_staircase_circuit_list = pm.run(opt_staircase_circuit_list)

In [ ]:
print('Random circuits')
print('# CX before:', opt_random_circuit_list[0].decompose().count_ops()['cx'], ', after:', opt_meas_random_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', opt_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  opt_meas_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Dense circuits')
print('# CX before:', opt_dense_circuit_list[0].decompose().count_ops()['cx'], ', after:', opt_meas_dense_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', opt_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after', opt_meas_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', opt_staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', opt_meas_staircase_circuit_list[0].decompose().count_ops()['cx'])
print('2Q depth before:', opt_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  opt_meas_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2))


In [ ]:
opt_meas_random_circuit_list[0].draw('mpl', idle_wires=False, fold=-1, scale=0.5)

In [ ]:
opt_meas_dense_circuit_list[0].draw('mpl', idle_wires=False, fold=-1, scale=0.5)

In [ ]:
opt_meas_staircase_circuit_list[0].draw('mpl', idle_wires=False, fold=-1, scale=0.5)

In [ ]:
backend.run(opt_meas_random_circuit_list[0])

## Use transpile function directly
Optimization level 3 implements the following passes:

- ContainsInstruction 
- UnitarySynthesis
- HighLevelSynthesis 
- BasisTranslator from source basis set() to target basis {'rz', 'delay', 'snapshot', 'sx', 'x', 'measure', 'ecr', 'barrier'}.
- OptimizeSwapBeforeMeasure 
- RemoveDiagonalGatesBeforeMeasure 
- SetLayout 
- VF2Layout 
- BarrierBeforeFinalMeasurements 
- SabreLayout 
- CheckMap 
- VF2PostLayout 
- ApplyLayout 
- UnitarySynthesis 
- HighLevelSynthesis 
- BasisTranslator from source basis {('swap', 2), ('ry', 1), ('cx', 2), ('rz', 1)} to target basis {'rz', 'delay', 'snapshot', 'sx', 'x', 'measure', 'ecr', 'barrier'}.
- CheckGateDirection
- GateDirection 
- Depth 
- Size 
- MinimumPoint 
- Collect2qBlocks 
- ConsolidateBlocks 
- UnitarySynthesis 
- Optimize1qGatesDecomposition 
- CommutationAnalysis 
- CommutativeCancellation 
- GatesInBasis 
- Depth 
- Size 
- MinimumPoint 
- Collect2qBlocks 
- ConsolidateBlocks 
- UnitarySynthesis 
- Optimize1qGatesDecomposition 
- CommutationAnalysis 
- CommutativeCancellation 
- GatesInBasis 
- Depth 
- Size 
- MinimumPoint 
- Collect2qBlocks 
- ConsolidateBlocks 
- UnitarySynthesis 
- Optimize1qGatesDecomposition 
- CommutationAnalysis 
- CommutativeCancellation 
- GatesInBasis 
- Depth 
- Size 
- MinimumPoint 
- Collect2qBlocks 
- ConsolidateBlocks 
- UnitarySynthesis 
- Optimize1qGatesDecomposition 
- CommutationAnalysis 
- CommutativeCancellation 
- GatesInBasis 
- Depth 
- Size 
- MinimumPoint 
- Collect2qBlocks 
- ConsolidateBlocks 
- UnitarySynthesis 
- Optimize1qGatesDecomposition 
- CommutationAnalysis 
- CommutativeCancellation 
- GatesInBasis 
- Depth 
- Size 
- MinimumPoint 
- ContainsInstruction 
- InstructionDurationCheck 
- ValidatePulseGates 



In [ ]:
from qiskit.compiler.transpiler import transpile

random_circuit_list_trans = transpile(random_circuit_list, backend=backend, optimization_level = 3, approximation_degree=0.99)
dense_circuit_list_trans = transpile(dense_layered_circuit_list, backend=backend, optimization_level = 3)
staircase_circuit_list_trans = transpile(staircase_circuit_list, backend=backend, optimization_level = 3)

In [ ]:
print('Random circuits')
print('# CX before:', opt_meas_random_circuit_list[0].decompose().count_ops()['cx'], ', after:', random_circuit_list_trans[0].decompose().count_ops()['cx'])
print('2Q depth before:', opt_meas_random_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  random_circuit_list_trans[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Dense circuits')
print('# CX before:', opt_meas_dense_circuit_list[0].decompose().count_ops()['cx'], ', after:', dense_circuit_list_trans[0].decompose().count_ops()['cx'])
print('2Q depth before:', opt_meas_dense_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after', dense_circuit_list_trans[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', opt_meas_staircase_circuit_list[0].decompose().count_ops()['cx'], ', after:', staircase_circuit_list_trans[0].decompose().count_ops()['cx'])
print('2Q depth before:', opt_meas_staircase_circuit_list[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  staircase_circuit_list_trans[0].decompose().depth(lambda x: x[0].num_qubits==2))


## Use AI transpiler service

In [ ]:
from qiskit_transpiler_service.transpiler_service import TranspilerService


In [ ]:
ai_cloud_transpiler_service = TranspilerService(
    backend_name='ibm_sherbrooke',
    ai=True,
    optimization_level=1,
)
random_circuit_list_ai_trans = ai_cloud_transpiler_service.run(random_circuit_list)
dense_circuit_list_ai_trans = ai_cloud_transpiler_service.run(dense_layered_circuit_list)
staircase_circuit_list_ai_trans = ai_cloud_transpiler_service.run(staircase_circuit_list)

In [ ]:
print('Random circuits')
print('# CX before:', random_circuit_list_trans[0].decompose().count_ops()['cx'], ', after:', random_circuit_list_ai_trans[0].decompose().count_ops()['cx'])
print('2Q depth before:', random_circuit_list_trans[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  random_circuit_list_ai_trans[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Dense circuits')
print('# CX before:', dense_circuit_list_trans[0].decompose().count_ops()['cx'], ', after:', dense_circuit_list_ai_trans[0].decompose().count_ops()['cx'])
print('2Q depth before:', dense_circuit_list_trans[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after', dense_circuit_list_ai_trans[0].decompose().depth(lambda x: x[0].num_qubits==2))

print('')
print('Staircase circuits')
print('# CX before:', staircase_circuit_list_trans[0].decompose().count_ops()['cx'], ', after:', staircase_circuit_list_ai_trans[0].decompose().count_ops()['cx'])
print('2Q depth before:', staircase_circuit_list_trans[0].decompose().depth(lambda x: x[0].num_qubits==2), ', after',  staircase_circuit_list_ai_trans[0].decompose().depth(lambda x: x[0].num_qubits==2))
